<a href="https://colab.research.google.com/github/mphani/ts_prophet/blob/works/actual_as_prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from prophet.plot import plot_plotly, plot_components_plotly

from prophet.plot import add_changepoints_to_plot, plot_cross_validation_metric


In [2]:
G_FILENAME = "sample_data/24122023_aerospike_stats.csv"

In [3]:
original_df = pd.read_csv(G_FILENAME)
original_df.head()

,cluster_name,instance,ns,service,metric_name,sub_query_name,prom_query,utc_date_time,metric_value
0,numa,192.168.200.239:9145,ssd,192.168.201.239:3000,aerospike_namespace_client_read_success,rate_query,rate(aerospike_namespace_client_read_success[1m]),1703010600,2882.088889
1,numa,192.168.200.239:9145,ssd,192.168.201.239:3000,aerospike_namespace_client_read_success,rate_query,rate(aerospike_namespace_client_read_success[1m]),1703010660,3006.044444
2,numa,192.168.200.239:9145,ssd,192.168.201.239:3000,aerospike_namespace_client_read_success,rate_query,rate(aerospike_namespace_client_read_success[1m]),1703010720,2960.888889
3,numa,192.168.200.239:9145,ssd,192.168.201.239:3000,aerospike_namespace_client_read_success,rate_query,rate(aerospike_namespace_client_read_success[1m]),1703010780,3010.822222
4,numa,192.168.200.239:9145,ssd,192.168.201.239:3000,aerospike_namespace_client_read_success,rate_query,rate(aerospike_namespace_client_read_success[1m]),1703010840,2992.222222


In [4]:
original_df["date_from_utc"] = pd.to_datetime(original_df.utc_date_time , unit='s')
original_df["full_date"] = original_df["date_from_utc"].dt.date
original_df["year"] = original_df["date_from_utc"].dt.year
original_df["month"] = original_df["date_from_utc"].dt.month
original_df["day"] = original_df["date_from_utc"].dt.day
original_df["hour"] = original_df["date_from_utc"].dt.hour
original_df["minute"] = original_df["date_from_utc"].dt.minute
original_df["second"] = original_df["date_from_utc"].dt.second

In [5]:
print( original_df.shape)

(419622, 17)


In [6]:
read_df= original_df[ original_df.metric_name=="aerospike_namespace_client_read_success"]
read_df= read_df[ read_df.sub_query_name=="simple_query"]
# pick up only ssd namespace
read_df= read_df[ read_df.ns=="ssd" ]
# read_df= read_df[ (read_df.ns=="ssd") & (read_df.service=="192.168.201.241:3000")]
read_df.shape

(69946, 17)

In [7]:
read_df= read_df[ read_df.service=="192.168.201.241:3000"]
read_df.shape

(23315, 17)

In [8]:
print(read_df["service"].unique())
print(read_df["ns"].unique())
print(read_df["cluster_name"].unique())
print(read_df["metric_name"].unique())

['192.168.201.241:3000']
['ssd']
['numa']
['aerospike_namespace_client_read_success']


In [14]:
read_df_len = len(read_df)

train_size = int(read_df_len * 0.8)

# 80% as train and 20% as test
#
train_read_df = read_df[:train_size]
test_df = read_df.iloc[train_size:]

print( "len(read_df_len)   ==> ",len(read_df))
print( "len(train_read_df) ==> ", len(train_read_df) )
print( "len(test_df)       ==> ", len(test_df) )

len(read_df_len)   ==>  23315
len(train_read_df) ==>  18652
len(test_df)       ==>  4663


In [ ]:
# read_df["full_date", "metric_value"].plot()
prophet_read_df = train_read_df[ ["date_from_utc", "metric_value"] ].copy()
prophet_read_df['date_from_utc'] = pd.to_datetime(prophet_read_df['date_from_utc'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
prophet_read_df.rename(columns={"date_from_utc": "ds", "metric_value": "y"}, inplace=True)
prophet_read_df=prophet_read_df.reindex()
prophet_read_df.info()

In [ ]:
l_sort_column_names = ["ds"]
prophet_read_df.sort_values(by= l_sort_column_names, ascending = True)

In [ ]:
# Removing duplicates in the original DataFrame
prophet_read_df.drop_duplicates(subset="ds", inplace=True)


In [ ]:
prophet_read_df.plot()

In [ ]:
# df['date_from_utc'] = pd.to_datetime(df['date_from_utc'])

# Plotting a line graph
plt.figure(figsize=(10, 6))  # Set figure size as needed

plt.plot(prophet_read_df['ds'], prophet_read_df['y'], color='blue', marker='o', linestyle='-')

# plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.title('Minute-wise Line Graph')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability if needed
plt.tight_layout()  # Adjusts plot to fit into the figure area
plt.show()

In [ ]:
# Plotting a line graph
plt.figure(figsize=(10, 6))  # Set figure size as needed

single_day_df = train_read_df [ train_read_df.day == 15 ]

single_day_df = single_day_df[ ["date_from_utc", "metric_value"] ].copy()
single_day_df.rename(columns={"date_from_utc": "ds", "metric_value": "y"}, inplace=True)
single_day_df.info()

# plt.plot(single_day_df['ds'], single_day_df['y'], color='green', marker='o', linestyle='--')
plt.bar (single_day_df['ds'], single_day_df['y'], )


plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.title('Minute-wise Line Graph')
# plt.xticks(rotation=45)  # Rotate x-axis labels for better readability if needed
plt.tight_layout()  # Adjusts plot to fit into the figure area
plt.show()

In [ ]:
m = Prophet()
# m= Prophet( interval_width=0.9, yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True )
m.fit(prophet_read_df)

In [ ]:
# 1440 = 24 * 60 minutes

#
# future = m.make_future_dataframe(periods= 1440, freq="T", include_history=True)
# future.tail()

In [ ]:
# forecast = m.predict(future)
#
# only for anamoly
forecast = m.predict( prophet_read_df)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
forecast.info()

In [ ]:
pd.concat( [prophet_read_df.set_index("ds")["y"],  forecast.set_index("ds")["yhat"]], axis=1).plot()

In [ ]:
# merge original and forecasted
#
merged_df = pd.merge( prophet_read_df, forecast[["ds", "yhat","yhat_lower","yhat_upper"]], on="ds" )

In [ ]:
merged_df.to_csv("prophet_forecasted.csv")

In [ ]:
figure_1 = m.plot(forecast)


In [ ]:
figure_2 = m.plot_components(forecast)

In [ ]:
plot_plotly(m, forecast)

In [ ]:
fig_change_points = add_changepoints_to_plot ( figure_1.gca(), m, forecast)
figure_1

In [ ]:
# error and metrics
perf_mae = mean_absolute_error(merged_df['y'], merged_df['yhat'])
print ("mae ", perf_mae)
perf_mape = mean_absolute_percentage_error(merged_df['y'], merged_df['yhat'])
print("mape " , perf_mape )

In [ ]:
merged_df["anomaly"] = merged_df.apply(lambda rows: 1 if ((rows.y<rows.yhat_lower)|(rows.y>rows.yhat_upper)) else 0, axis = 1)

# Check the number of anomalies
merged_df["anomaly"].value_counts()

In [ ]:
anomalies_df = merged_df[ merged_df["anomaly"]==1].sort_values(by='ds')
anomalies_df.head(100)

In [ ]:
# Visualize the anomalies
plt.figure(figsize=(10, 6))
sns.scatterplot(x="ds", y="y", data=merged_df, hue="anomaly")
sns.lineplot(x="ds", y="yhat", data=merged_df, color="black")

In [ ]:
# Calculate residuals (difference between actual and predicted values)
merged_df['residuals'] = merged_df['y'] - merged_df['yhat']

merged_df.info()

# single_merged_df =


In [ ]:
# Calculate mean and standard deviation of residuals
mean_residual = merged_df['residuals'].mean()
std_residual = merged_df['residuals'].std()

# Set a threshold for anomalies (e.g., 3 standard deviations)
threshold = 3 * std_residual

# Identify anomalies
anomalies = merged_df[np.abs(merged_df['residuals'] - mean_residual) > threshold]

# Plotting the data and anomalies
plt.figure(figsize=(10, 6))
plt.plot(merged_df['ds'], merged_df['y'], label='Original Data')
plt.scatter(anomalies['ds'], anomalies['y'], color='red', label='Anomalies')
plt.legend()
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.title('Anomaly Detection with Prophet')
plt.show()

In [ ]:
perf_cv = cross_validation(m, initial='6 days', period='2 days', horizon = '1 days')

In [ ]:
perf_cv_metrics = performance_metrics( perf_cv)
perf_cv_metrics.info()

perf_cv_metrics

In [ ]:
fig = plot_cross_validation_metric(perf_cv, metric='mape')